In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import tensorrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import LSTM, Dense, Input

In [ ]:
raw_df = pd.read_csv('car_data.csv')
print(raw_df.shape)
raw_df.head()

# Good performances with:

LSTM : 16/32/16 | Timesteps 32 | Lookahead 1 | Batch Size : 256 | Dropout X

In [ ]:
def SoC_forecasting_model(timesteps, features, batch_size, output_shape, dropout=0, recurrent_dropout=0, verbose=True):
    """
    - cuDNN Requirements
    activation == tanh
    recurrent_activation == sigmoid
    recurrent_dropout == 0
    unroll is False
    use_bias is True
    - All those are default values
    """
    STATEFUL = True
    SoC_model = Sequential(name ='SoC_Forecasting_Model')
    SoC_model.add(Input(shape=(timesteps, features), batch_size=batch_size, name='Input'))
    SoC_model.add(LSTM(16, dropout=dropout, recurrent_dropout=recurrent_dropout, stateful=STATEFUL, name='LSTM_1', return_sequences=True))
    SoC_model.add(LSTM(32, dropout=dropout, recurrent_dropout=recurrent_dropout, stateful=STATEFUL, name='LSTM_2', return_sequences=True))
    SoC_model.add(LSTM(16, dropout=dropout, recurrent_dropout=recurrent_dropout, stateful=STATEFUL, name='LSTM_last'))
    SoC_model.add(Dense(output_shape, name='Dense_output'))
    SoC_model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    if verbose :
        print(SoC_model.summary())
    return SoC_model


In [ ]:
def AIO_preprocess_train_SoC_model2(raw_df, look_back=32, look_ahead=1, batch_size=256, test_size=0.2, epochs=20, verbose=True):

    df = raw_df.copy()

    scaler_features = MinMaxScaler()
    scaler_voltage = MinMaxScaler()

    # Pre-process of data
    # Reduce noise in voltage measures
    # Transform timestamps into cycles number
    THRESHOLD = 1000
    df['diff_time'] = df['timestamp'].diff()
    df['diff_time'] = df['diff_time'].abs()
    df['cycle'] = (df['diff_time'] > THRESHOLD).cumsum()
    df['timestamp'] = df['timestamp'] - df.groupby('cycle')['timestamp'].transform('min')
    df['voltage_smoothed'] = df['voltage'].ewm(alpha=0.1, adjust=False).mean()
    df['voltage_smoothed'] = df['voltage_smoothed'].fillna(df['voltage'])
    df.drop(['voltage','timestamp','diff_time'], axis=1, inplace=True)

    # Split into features and target
    # Scale features and target
    df_voltage = df['voltage_smoothed']
    df_cycle = df['cycle']
    df.drop(['cycle'], axis=1, inplace=True)
    columns = df.columns.tolist()

    df = pd.DataFrame(scaler_features.fit_transform(df), columns=columns)
    df_voltage = pd.DataFrame(scaler_voltage.fit_transform(df_voltage.values.reshape(-1,1)), columns=['voltage'])

    df = pd.concat([df_cycle, df], axis=1)
    df_voltage = pd.concat([df_cycle, df_voltage], axis=1)

    X_train, X_test, Y_train, Y_test = train_test_split(np.array(df), np.array(df_voltage), test_size=test_size, shuffle=False)
    columns.insert(0,'cycle')
    X_train = pd.DataFrame(X_train, columns=columns)
    X_test = pd.DataFrame(X_test, columns=columns)
    Y_train = pd.DataFrame(Y_train, columns=['cycle','voltage'])
    Y_test = pd.DataFrame(Y_test, columns=['cycle','voltage'])

    X_train_gp = X_train.groupby('cycle')
    Y_train_gp = Y_train.groupby('cycle')
    mse_train = []
    mae_train = []

    if verbose :
        print('Training set size  :',X_train.shape[0])
        print('Test set size      :',X_test.shape[0])

    model = SoC_forecasting_model(look_back, X_train.shape[1]-1, batch_size, look_ahead, verbose=verbose)



    if verbose :
        print(f'{"TRAINING":-^100}')
    for epoch in range(epochs):
        for i in X_train_gp.groups.keys():

            X = np.delete(X_train_gp.get_group(i).values, 0, axis=1)
            Y = np.delete(Y_train_gp.get_group(i).values, 0, axis=1)
            Y = np.append(Y, np.zeros(look_ahead)) # Padding
            X_batchs = []
            Y_batchs = []
            for j in range(look_back, len(X)):
                X_batchs.append(X[j-look_back:j])
                Y_batchs.append(Y[j:j+look_ahead])
            X_batchs = np.array(X_batchs)
            Y_batchs = np.array(Y_batchs)
            for k in range(len(X_batchs)//batch_size):
                X_batch = X_batchs[k*batch_size : (k+1)*batch_size]
                Y_batch = Y_batchs[k*batch_size : (k+1)*batch_size]
                metrics = model.train_on_batch(X_batch, Y_batch, reset_metrics=False)
                mse_train.append(metrics[0])
                mae_train.append(metrics[1])

            model.reset_states()
    plt.plot(mse_train, label='mse')
    plt.plot(mae_train, label='mae')
    plt.legend()
    plt.show()

    return model, scaler_features, scaler_voltage, X_train, X_test, Y_train, Y_test

model, scaler_X, scaler_Y, X_train, X_test, Y_train, Y_test = AIO_preprocess_train_SoC_model2(raw_df)

In [ ]:
def AIO_preprocess_test_SoC_model2(model, X_test, Y_test, look_back=32, look_ahead=1, batch_size=256, verbose=True):

    X_test_gp = X_test.groupby('cycle')
    Y_test_gp = Y_test.groupby('cycle')
    mse_test = []
    mae_test = []

    if verbose :
        print(f'{"TESTING":-^100}')
    for i in X_test_gp.groups.keys():
        X = np.delete(X_test_gp.get_group(i).values, 0, axis=1)
        Y = np.delete(Y_test_gp.get_group(i).values, 0, axis=1)
        Y = np.append(Y, np.zeros(look_ahead)) # Padding
        X_batchs = []
        Y_batchs = []
        for j in range(look_back, len(X)):
            X_batchs.append(X[j-look_back:j])
            Y_batchs.append(Y[j:j+look_ahead])
        X_batchs = np.array(X_batchs)
        Y_batchs = np.array(Y_batchs)
        for k in range(len(X_batchs)//batch_size):
            X_batch = X_batchs[k*batch_size : (k+1)*batch_size]
            Y_batch = Y_batchs[k*batch_size : (k+1)*batch_size]
            metrics = model.test_on_batch(X_batch, Y_batch, reset_metrics=False)
            mse_test.append(metrics[0])
            mae_test.append(metrics[1])

    plt.plot(mse_test, label='mse')
    plt.plot(mae_test, label='mae')
    plt.legend()
    plt.show()

    return mse_test, mae_test

mse_test, mae_test = AIO_preprocess_test_SoC_model2(model, X_test, Y_test)

In [ ]:
def predict(model, X_input):
    pass

In [ ]:
def AIO_preprocess_train_SoC_model(df, look_back=32, look_ahead=1, test_size=0.2, epochs=20, batch_size=256, verbose=True, evaluate=True):

    scaler_X = MinMaxScaler()
    scaler_Y = MinMaxScaler()

    df.astype(float)

    df['voltage_smoothed'] = df['voltage'].ewm(alpha=0.1, adjust=False).mean()
    df['voltage_smoothed'] = df['voltage_smoothed'].fillna(df['voltage'])

    raw_X = df.drop(['timestamp', 'voltage'], axis=1)
    raw_Y = np.array(df['voltage_smoothed'])

    raw_X = scaler_X.fit_transform(raw_X)
    raw_Y = scaler_Y.fit_transform(raw_Y.reshape(-1,1))
    raw_Y = raw_Y.reshape(-1)

    X_train, X_test, Y_train, Y_test = train_test_split(np.array(raw_X), np.array(raw_Y), test_size=test_size, shuffle=False)
    model = SoC_forecasting_model(look_back, X_train.shape[1], batch_size, look_ahead, dropout=0, verbose=verbose)
    if verbose :
        print('X_train  :',X_train.shape)
        print('Y_train  :',Y_train.shape)
        print('X_test   :',X_test.shape)
        print('Y_test   :',Y_test.shape)

    mse_train = []
    mae_train = []
    mse_test = []
    mae_test = []
    if verbose :
        print(f'{"TRAINING":-^100}')
    for i in range(epochs):
        for j in tqdm(  range((X_train.shape[0]-look_ahead-look_back)//batch_size)  , desc=f"Epoch: {i+1:0>{len(str(epochs))}}/{epochs:0>{len(str(epochs))}}", colour='green', disable=not verbose):
            X_batch = np.empty((batch_size, look_back, X_train.shape[1]))
            Y_batch = np.empty((batch_size, look_ahead))
            for k in range(j*batch_size, (j+1)*batch_size):
                X_batch[k - j*batch_size] = X_train[k : k + look_back]
                Y_batch[k - j*batch_size] = Y_train[k + look_back : k + look_back + look_ahead]
            metrics = model.train_on_batch(X_batch, Y_batch, reset_metrics=False)
            mse_train.append(metrics[0])
            mae_train.append(metrics[1])
        model.reset_states()

    if verbose :
        plt.plot(mse_train, label='mse')
        plt.plot(mae_train, label='mae')
        plt.title('Training stats')
        plt.legend()
        plt.show()

    if evaluate :
        if verbose :
            print(f'{"EVALUATION":-^100}')
        for j in tqdm(range((X_test.shape[0]-look_ahead-look_back)//batch_size), desc='Evaluation', colour='red', disable=not verbose):
            X_batch = np.empty((batch_size, look_back, X_test.shape[1]))
            Y_batch = np.empty((batch_size, look_ahead))
            for k in range(j*batch_size, (j+1)*batch_size):
                X_batch[k - j*batch_size] = X_test[k : k + look_back]
                Y_batch[k - j*batch_size] = Y_test[k + look_back : k + look_back + look_ahead]
            metrics = model.test_on_batch(X_batch, Y_batch, reset_metrics=False)
            mse_test.append(metrics[0])
            mae_test.append(metrics[1])
        if verbose :
            plt.plot(mse_test, label='mse')
            plt.plot(mae_test, label='mae')
            plt.title('Testing stats')
            plt.legend()
            plt.show()

    return model, scaler_X, scaler_Y, X_train, X_test, Y_train, Y_test

model, scaler_X, scaler_Y, X_train, X_test, Y_train, Y_test = AIO_preprocess_train_SoC_model(raw_df)

In [ ]:
res =[]
look_back=32
look_ahead=1
batch_size=256
for j in tqdm(range((X_test.shape[0]-look_back-look_ahead)//batch_size), ncols=100, desc='Prediction', colour='blue'):
    X_batch = np.empty((batch_size, look_back, X_test.shape[1]))
    for k in range(j*batch_size, (j+1)*batch_size):
        X_batch[k - j*batch_size] = X_test[k : k + look_back]
    Y_batch = model.predict_on_batch(X_batch)
    res.extend(Y_batch.reshape(-1))
plt.plot(res, label='prediction')
plt.plot(Y_test, label='real')
plt.legend()
plt.show()
